# Assignment 2

2.2

Write a DES program to verify this for n=1, n=2 and n=4. Make sure that your result
has a high and known statistical significance. How does the number of measurements
required to attain this, depend on $\rho$?

In [3]:
import random
import simpy
import numpy as np
import pandas as pd
from IPython.display import display, HTML

In [52]:
# Init variables
RANDOM_SEED = 42
N_helpers = [1, 2, 4]  # Number of machines in the queue
serveTime = 6          # Minutes it takes to help a customer
lambdaIAT = 6          # Create a customer every ~7 minutes
SIM_TIME = 80          # Simulation time in minutes
NRUNS = 100             # Amount of runs
customerCount = 50   # Amount of customers

# Choose your queueing system
# resources = [simpy.PriorityResource]
resources = [simpy.Resource]
# resources = [simpy.Resource, simpy.PriorityResource]

# Create Queue object
class Queue(object):
    def __init__(self, env, N, serveTime, resource):
        self.env = env
        self.machine = resource(env, N)
        self.serveTime = serveTime
        self.customerHelped = 0
        self.helperN = N

    def helped(self, customer, customerServeWait=serveTime):
        yield self.env.timeout(customerServeWait)

# Customer with its own serviceTime
def customer(env, name, cw, id, customerServeWait, resource):
    customerServeWait = random.randint(serveTime - 5, serveTime + 5)
    enterQueue = env.now
    if resource == simpy.PriorityResource:
        with cw.machine.request(priority=customerServeWait) as request:
            request.name = name
            request.time = customerServeWait
            yield request

            customerStat[cw.helperN].append(env.now - enterQueue)
            yield env.process(cw.helped(name, customerServeWait=customerServeWait))

    elif resource == simpy.Resource:
        with cw.machine.request() as request:
            request.name = name
            request.time = customerServeWait
            yield request

            customerStat[cw.helperN].append(env.now - enterQueue)
            yield env.process(cw.helped(name, customerServeWait=customerServeWait))


def setup(env, N, serveTime, lambdaIAT, customerCount, resource):
    queue = Queue(env, N, serveTime, resource)

    # Create more customers while the simulation is running
    while queue.customerHelped < customerCount:
        s = np.random.poisson(lambdaIAT, customerCount)
        yield env.timeout(s[queue.customerHelped])
        queue.customerHelped += 1
        env.process(
            customer(env, 'customer %d' % queue.customerHelped, queue,
                     queue.customerHelped,
                     (customerCount - queue.customerHelped), resource))

for resource in resources:
    random.seed(RANDOM_SEED)
    customerStat = {}
    for j in range(NRUNS):
        for N in N_helpers:
            customerStat.setdefault(N, [])
            customerStat.setdefault(str(N) + "P", [])
            customerStat[str(N) + "P"].append(lambdaIAT / (N * serveTime))
            env = simpy.Environment()
            env.process(
                setup(env, N, serveTime, lambdaIAT, customerCount, resource))
            env.run()
    
    # Format data    
    customerStatDf = pd.DataFrame({"Helpers": N_helpers, 
                                  "$\rho$": [np.mean(customerStat[str(N)+'P']).round(decimals=2) for N in N_helpers],
                                  "Average": [np.mean(customerStat[N]).round(decimals=2) for N in N_helpers], 
                                  "Variance": [np.var(customerStat[N]).round(decimals=2) for N in N_helpers],
                                  "std.dev": [np.std(customerStat[N]).round(decimals=2) for N in N_helpers]}).set_index('Helpers')
    
    # Print output    
    print(f"For {str(resource)[33:-2]}")
    display(HTML(customerStatDf.T.to_html()))
    print()


For Resource


Helpers,1,2,4
$\rho$,1.00,0.50,0.25
Average,14.62,0.11,0.00
Variance,186.39,0.29,0.00
std.dev,13.65,0.54,0.02


## 2.3

Also compare the result to that for an M/M/1 queue with shortest job first scheduling,
where you always give priority to the smallest jobs.

In [53]:
# Init variables
RANDOM_SEED = 42
N_helpers = [1, 2, 4]  # Number of machines in the queue
waitTime = 6          # Minutes it takes to help a customer
lambdaIAT = 6          # Create a customer every ~7 minutes
SIM_TIME = 80          # Simulation time in minutes
NRUNS = 10             # Amount of runs
customerCount = 1000   # Amount of customers

# Choose your queueing system
# resources = [simpy.PriorityResource]
# resources = [simpy.Resource]
resources = [simpy.Resource, simpy.PriorityResource]


class Queue(object):
    def __init__(self, env, N, waitTime, resource):
        self.env = env
        self.machine = resource(env, N)
        self.waitTime = waitTime
        self.customerHelped = 0
        self.helperN = N

    def helped(self, customer, customerWait=waitTime):
        yield self.env.timeout(customerWait)


def customer(env, name, cw, id, customerWait, resource):
    customerWait = random.randint(waitTime - 5, waitTime + 5)
    enterQueue = env.now
    if resource == simpy.PriorityResource:
        with cw.machine.request(priority=customerWait) as request:
            request.name = name
            request.time = customerWait
            yield request

            customerStat[cw.helperN].append(env.now - enterQueue)
            yield env.process(cw.helped(name, customerWait=customerWait))

    elif resource == simpy.Resource:
        with cw.machine.request() as request:
            request.name = name
            request.time = customerWait
            yield request

            customerStat[cw.helperN].append(env.now - enterQueue)
            yield env.process(cw.helped(name, customerWait=customerWait))


def setup(env, N, waitTime, lambdaIAT, customerCount, resource):
    queue = Queue(env, N, waitTime, resource)

    # Create more customers while the simulation is running
    while queue.customerHelped < customerCount:
        s = np.random.poisson(lambdaIAT, customerCount)
        yield env.timeout(s[queue.customerHelped])
        queue.customerHelped += 1
        env.process(
            customer(env, 'customer %d' % queue.customerHelped, queue,
                     queue.customerHelped,
                     (customerCount - queue.customerHelped), resource))

for resource in resources:
    random.seed(RANDOM_SEED)
    customerStat = {}
    for j in range(NRUNS):
        for N in N_helpers:
            #         print(f"******** N={N} Helpers ********")
            customerStat.setdefault(N, [])
            customerStat.setdefault(str(N) + "P", [])
            customerStat[str(N) + "P"].append(lambdaIAT / (N * waitTime))
            env = simpy.Environment()
            env.process(
                setup(env, N, waitTime, lambdaIAT, customerCount, resource))
            env.run()
    customerStatDf = pd.DataFrame({"Helpers": N_helpers, 
                                  "$\rho$": [np.mean(customerStat[str(N)+'P']).round(decimals=2) for N in N_helpers],
                                  "Average": [np.mean(customerStat[N]).round(decimals=2) for N in N_helpers], 
                                  "Variance": [np.var(customerStat[N]).round(decimals=2) for N in N_helpers],
                                  "std.dev": [np.std(customerStat[N]).round(decimals=2) for N in N_helpers]}).set_index('Helpers')
    print(f"For {str(resource)[33:-2]}")
    display(HTML(customerStatDf.T.to_html()))
    print()


For Resource


Helpers,1,2,4
$\rho$,1.00,0.50,0.25
Average,57.90,0.15,0.00
Variance,2888.22,0.44,0.00
std.dev,53.74,0.66,0.00



For PriorityResource


Helpers,1,2,4
$\rho$,1.00,0.50,0.25
Average,41.56,0.15,0.00
Variance,23678.32,0.47,0.00
std.dev,153.88,0.68,0.00


## 2.4

Now experiment with different service rate distributions. On the one hand try the
M/D/1 and M/D/n queues, on the other hand try a long-tail distribution. For the latter
you may e.g. use a distribution where 75% of the jobs have an exponential distribution
with an average service time of 1.0 and the remaining 25% an exponential distribution
with an average service time of 5.0 (an example of a hyperexponential distribution).

In [6]:
# Init variables
RANDOM_SEED = 42
N_helpers = [1, 2, 4]  # Number of machines in the queue
waitTime = 6           # Minutes it takes to help a customer
lambdaIAT = 6          # Create a customer every ~7 minutes
serviceScheme = "Long-Tailed" # Can be "Long-Tailed", "Poisson", "Deterministic", "Inverse"
serviceSchemes = ["Long-Tailed", "Poisson", "Deterministic", "Inverse"]
ltLow = 1              # Lower bound long-tailed distribution
ltHigh = 5             # Upper bound long-tailed distribution
SIM_TIME = 80          # Simulation time in minutes
NRUNS = 10             # Amount of runs
customerCount = 1000   # Amount of customers


# Choose your queueing system
# resources = [simpy.PriorityResource]
# resources = [simpy.Resource]
resources = [simpy.Resource, simpy.PriorityResource]


class Queue(object):
    def __init__(self, env, N, waitTime, resource):
        self.env = env
        self.machine = resource(env, N)
        self.waitTime = waitTime
        self.customerHelped = 0
        self.helperN = N

    def helped(self, customer, customerWait=waitTime):
        yield self.env.timeout(customerWait)


def customer(env, name, cw, id, customerWait, resource, customerScheme):
    if serviceScheme == "Long-Tailed":
        rCheck = random.random()
        if rCheck <= 0.75:
            customerWait = ltLow
        else:
            customerWait = ltHigh
    elif serviceScheme == "Poisson":
        customerWait = random.randint(waitTime - 5, waitTime + 5)
    elif serviceScheme == "Deterministic":
        customerWait = waitTime
    elif serviceScheme == "Inverse":
        customerWait = customerWait
        
    
    enterQueue = env.now
    if resource == simpy.PriorityResource:
        with cw.machine.request(priority=customerWait) as request:
            request.name = name
            request.time = customerWait
            yield request

            customerStat[cw.helperN].append(env.now - enterQueue)
            yield env.process(cw.helped(name, customerWait=customerWait))

    elif resource == simpy.Resource:
        with cw.machine.request() as request:
            request.name = name
            request.time = customerWait
            yield request

            customerStat[cw.helperN].append(env.now - enterQueue)
            yield env.process(cw.helped(name, customerWait=customerWait))


def setup(env, N, waitTime, lambdaIAT, customerCount, resource, customerScheme):
    queue = Queue(env, N, waitTime, resource)

    # Create more customers while the simulation is running
    while queue.customerHelped < customerCount:
        s = np.random.poisson(lambdaIAT, customerCount)
        yield env.timeout(s[queue.customerHelped])
        queue.customerHelped += 1
        env.process(
            customer(env, 'customer %d' % queue.customerHelped, queue,
                     queue.customerHelped,
                     (customerCount - queue.customerHelped), resource, customerScheme))

for customerScheme in customerSchemes:
    for resource in resources:
        random.seed(RANDOM_SEED)
        customerStat = {}
        for j in range(NRUNS):
            for N in N_helpers:
                #         print(f"******** N={N} Helpers ********")
                customerStat.setdefault(N, [])
                customerStat.setdefault(str(N) + "P", [])
                customerStat[str(N) + "P"].append(lambdaIAT / (N * waitTime))
                env = simpy.Environment()
                env.process(
                    setup(env, N, waitTime, lambdaIAT, customerCount, resource, customerScheme))
                env.run()
        customerStatDf = pd.DataFrame({"Helpers": N_helpers, 
                                      "$\rho$": [np.mean(customerStat[str(N)+'P']).round(decimals=2) for N in N_helpers],
                                      "Average": [np.mean(customerStat[N]).round(decimals=2) for N in N_helpers], 
                                      "Variance": [np.var(customerStat[N]).round(decimals=2) for N in N_helpers],
                                      "std.dev": [np.std(customerStat[N]).round(decimals=2) for N in N_helpers]}).set_index('Helpers')
        print(f"For {str(resource)[33:-2]} with customerScheme = {customerScheme}")
        display(HTML(customerStatDf.T.to_html()))
        print()


NameError: name 'customerSchemes' is not defined